In [ ]:
import io
import numpy as np
import pandas as pd
import math

In [ ]:
dados = pd.read_csv("/content/dataset.csv")
segmentos_enlaces = pd.read_csv("/content/dataset-segmentos.csv")

In [ ]:
def dbm2mw(v):
  return 10 ** (v/10)
def mw2dbm(v):
  return 10 * np.log10(v)

def potenciaReal(row):
  if (row['sf'] == 10 and row['pw'] == 20): return 7.50;
  elif (row['sf'] == 9 and row['pw'] == 20): return 8.15;
  elif (row['sf'] == 8 and row['pw'] == 20): return 8.69;
  elif (row['sf'] == 7 and row['pw'] == 20): return 13.64;

  elif (row['sf'] == 10 and row['pw'] == 17): return 6.64;
  elif (row['sf'] == 9 and row['pw'] == 17): return 7.48;
  elif (row['sf'] == 8 and row['pw'] == 17): return 8.01;
  elif (row['sf'] == 7 and row['pw'] == 17): return 12.77;

  elif (row['sf'] == 10 and row['pw'] == 14): return 4.87;
  elif (row['sf'] == 9 and row['pw'] == 14): return 5.08;
  elif (row['sf'] == 8 and row['pw'] == 14): return 5.91;
  elif (row['sf'] == 7 and row['pw'] == 14): return 9.50;

  elif (row['sf'] == 10 and row['pw'] == 11): return 2.42;
  elif (row['sf'] == 9 and row['pw'] == 11): return 2.87;
  elif (row['sf'] == 8 and row['pw'] == 11): return 3.58;
  elif (row['sf'] == 7 and row['pw'] == 11): return 7.33;


def fatorCarrecao(row):
  if (row['sf'] == 10 and row['pw'] == 20): return 3.35;
  elif (row['sf'] == 9 and row['pw'] == 20): return 3.34;
  elif (row['sf'] == 8 and row['pw'] == 20): return 3.42;
  elif (row['sf'] == 7 and row['pw'] == 20): return 3.50;

  elif (row['sf'] == 10 and row['pw'] == 17): return 3.86;
  elif (row['sf'] == 9 and row['pw'] == 17): return 4.07;
  elif (row['sf'] == 8 and row['pw'] == 17): return 4.05;
  elif (row['sf'] == 7 and row['pw'] == 17): return 3.85;

  elif (row['sf'] == 10 and row['pw'] == 14): return 6.23;
  elif (row['sf'] == 9 and row['pw'] == 14): return 6.40;
  elif (row['sf'] == 8 and row['pw'] == 14): return 6.68;
  elif (row['sf'] == 7 and row['pw'] == 14): return 4.50;

  elif (row['sf'] == 10 and row['pw'] == 11): return 3.61;
  elif (row['sf'] == 9 and row['pw'] == 11): return 3.74;
  elif (row['sf'] == 8 and row['pw'] == 11): return 3.60;
  elif (row['sf'] == 7 and row['pw'] == 11): return 3.59;

In [ ]:
def fspl(row):
  return round(20 * math.log10(row['distancia']) + 20 * math.log10(row['freq'] * 10 ** 6) + 20 * math.log10((4 * math.pi) / 299792458), 2)

In [ ]:
posicoes = dados.groupby(['id','data','origem_antena','origem_altura','destino_antena','destino_altura','origem_lat','origem_lng','destino_lat','destino_lng','distancia','freq','pw','sf','cr','cobertura'])
mean = posicoes['rssi'].mean().round(2).reset_index(name='RSSI')

mean['potenciaReal'] = mean.apply(lambda row: potenciaReal(row), axis=1)
mean['SNR'] = posicoes['snr'].mean().round(2).reset_index(name='SNR')['SNR']
mean['EIRP'] = mean['potenciaReal'] + mean['origem_antena']
mean['IRL'] = mean['RSSI'] + mean['destino_antena']
mean['PL'] = mean['EIRP'] - mean['IRL']
mean['FSPL'] = mean.apply(lambda row: fspl(row), axis=1)
mean['FC'] = mean.apply(lambda row: fatorCarrecao(row), axis=1)
mean['PL-FC'] = mean['PL'] - mean['FC']


In [ ]:
print('Total de registros:', dados['cobertura'].count())
print(dados.groupby(['cobertura'])['cobertura'].count())

Total de registros: 9730
cobertura
Floresta     530
Mista        318
Praia       7884
Rua          998
Name: cobertura, dtype: int64


In [ ]:
from scipy.optimize import curve_fit
import warnings
warnings.simplefilter("ignore")

def objctiveFunctionLD(d, A, n, d0):
    if d0 <= 0: d0 = 0.000001
    return A + 10 * n * np.log10(d/d0)

valores_n = pd.DataFrame({'cobertura': [], 'pw': [], 'sf': [], 'PLd0':[], 'd0':[], 'n':[]})
valores_n = valores_n.astype({'cobertura': str,'pw': int,'sf':int, 'PLd0':float,'d0':float,'n':float})

coberturas = ['Praia', 'Rua', 'Floresta', 'Mista']
pws = [20,17,14,11]
sfs = [10,9,8,7]
for cobertura in coberturas:
  for pw in pws:
    for sf in sfs:

      if (cobertura != 'Mista'):
        selecionados = (mean['pw'] == pw) & (mean['sf'] == sf) & (mean['cobertura'] == cobertura);

      if (cobertura == 'Mista'):
        selecionados = (mean['pw'] == pw) & (mean['sf'] == sf) & (mean['cobertura'] != 'Praia')

      params, _ = curve_fit(objctiveFunctionLD, mean[selecionados]['distancia'], mean[selecionados]['PL-FC'], p0=[31.56, 2, 1])
      PLd0, n, d0 = params

      novo_n = pd.DataFrame({
          'cobertura': cobertura,
          'pw': pw,
          'sf': sf,
          'PLd0': round(PLd0, 2),
          'd0': round(d0, 2),
          'n': round(n, 2)
      }, index=[0])

      valores_n = pd.concat([valores_n, novo_n], ignore_index=True)

print(valores_n)

   cobertura  pw  sf   PLd0    d0     n
0      Praia  20  10  27.01  1.59  2.35
1      Praia  20   9  24.54  1.20  2.33
2      Praia  20   8  30.60  2.28  2.35
3      Praia  20   7  26.93  1.44  2.39
4      Praia  17  10  35.89  3.61  2.32
..       ...  ..  ..    ...   ...   ...
59     Mista  14   7  15.05  3.02  3.46
60     Mista  11  10  17.73  2.20  3.04
61     Mista  11   9  14.26  1.79  3.07
62     Mista  11   8  19.40  2.57  3.08
63     Mista  11   7   9.36  1.76  3.42

[64 rows x 6 columns]


In [ ]:
def log_distance(row, valores_n):
    selecionados = ((valores_n['pw'] == row['pw']) & (valores_n['sf'] == row['sf']) & (valores_n['cobertura'] == row['cobertura']))

    if selecionados.any():
        n = valores_n.loc[selecionados, 'n'].values[0]
        d0 = valores_n.loc[selecionados, 'd0'].values[0]
        PLd0 = valores_n.loc[selecionados, 'PLd0'].values[0]

        return (PLd0 + 10 * n * math.log10(row['distancia'] / d0))
    else:
        return None

mean['Log-Distance'] = mean.apply(lambda row: log_distance(row, valores_n), axis=1)


In [ ]:
def log_distance_ME(row, valores_n, segmentos_enlaces):
  segmentos = segmentos_enlaces['posicao_id'] == row['id']
  if segmentos.any():
    d_segmentos = segmentos_enlaces.loc[segmentos, ['distancia' , 'cobertura']]
  else:
    d_segmentos = pd.DataFrame({'distancia': [row['distancia']], 'cobertura': [row['cobertura']]})

  PL = d0 = None
  dSegmento = 0
  for segmento in d_segmentos.itertuples(index=False):
    selecionados = ((valores_n['pw'] == row['pw']) & (valores_n['sf'] == row['sf']) & (valores_n['cobertura'] == segmento[1]))
    if selecionados.any():
      n = valores_n.loc[selecionados, 'n'].values[0]

      if PL == None:
        PL = valores_n.loc[selecionados, 'PLd0'].values[0]
        d0 = valores_n.loc[selecionados, 'd0'].values[0]

      dSegmento += segmento[0]

      PL = PL + (10 * n * np.log10((dSegmento)/d0))
      d0 += segmento[0]
  return PL

mean['log_distance_ME'] = mean.apply(lambda row: log_distance_ME(row, valores_n, segmentos_enlaces), axis=1)

In [ ]:
def maxCover(row, valores_n, segmentos_enlaces):
  segmentos = segmentos_enlaces['posicao_id'] == row['id']
  if segmentos.any():
    d_segmentos = segmentos_enlaces.loc[segmentos, ['distancia' , 'cobertura']]
  else:
    d_segmentos = pd.DataFrame({'distancia': [row['distancia']], 'cobertura': [row['cobertura']]})

  wavelength = 299792458 / row['freq']
  K = dbm2mw(row['origem_antena']) * dbm2mw(row['destino_antena']) * ((wavelength / (4 * math.pi)) ** 2)

  produtorio = 1

  for segmento in d_segmentos.itertuples(index=False):
    selecionados = ((valores_n['pw'] == row['pw']) & (valores_n['sf'] == row['sf']) & (valores_n['cobertura'] == segmento[1]))
    if selecionados.any():
      n = valores_n.loc[selecionados, 'n'].values[0]
      produtorio *= (1 / (segmento[0] ** n));

  Pr = mw2dbm(K * dbm2mw(row['potenciaReal']) * produtorio)

  IRL = Pr + row['destino_antena']
  EIRP = row['potenciaReal'] + row['origem_antena']
  PL = EIRP - IRL

  return PL

mean['maxCover'] = mean.apply(lambda row: maxCover(row, valores_n, segmentos_enlaces), axis=1)

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

desempenho_modelos = pd.DataFrame({'modelo': [], 'MAE': [], 'MAPE':[], 'MSE':[], 'RMSE':[]})

enlaces_cobertura_unica = mean[(mean['cobertura'] != 'Mista')]

desempenho_modelos = pd.concat([desempenho_modelos,  pd.DataFrame({
    'modelo': 'FSPL',
    'MAE': round(mean_absolute_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['FSPL']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['FSPL']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['FSPL']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['FSPL'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos = pd.concat([desempenho_modelos, pd.DataFrame({
    'modelo': 'Log-Distance',
    'MAE': round(mean_absolute_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['Log-Distance'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos = pd.concat([desempenho_modelos, pd.DataFrame({
    'modelo': 'Log-Distance Multi-Exponent',
    'MAE': round(mean_absolute_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['log_distance_ME']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['log_distance_ME']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['log_distance_ME']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_unica['PL-FC'], enlaces_cobertura_unica['log_distance_ME'])), 2)
}, index=[0])], ignore_index=True)

print(desempenho_modelos)

                        modelo   MAE  MAPE    MSE  RMSE
0                         FSPL  5.72  0.11  42.13  6.49
1                 Log-Distance  2.00  0.03   6.67  2.58
2  Log-Distance Multi-Exponent  2.00  0.03   6.67  2.58


In [ ]:
enlaces_ambiente = mean[(mean['cobertura'] != 'Praia')]
enlaces_cobertura_mista = mean[(mean['cobertura'] == 'Mista')]

desempenho_modelos_CM = pd.DataFrame({'modelo': [], 'MAE': [], 'MAPE':[], 'MSE':[], 'RMSE':[]})

desempenho_modelos_CM = pd.concat([desempenho_modelos_CM,  pd.DataFrame({
    'modelo': 'FSPL',
    'MAE': round(mean_absolute_error(enlaces_ambiente['PL-FC'], enlaces_ambiente['FSPL']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_ambiente['PL-FC'], enlaces_ambiente['FSPL']), 2),
    'MSE': round(mean_squared_error(enlaces_ambiente['PL-FC'], enlaces_ambiente['FSPL']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_ambiente['PL-FC'], enlaces_ambiente['FSPL'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos_CM = pd.concat([desempenho_modelos_CM, pd.DataFrame({
    'modelo': 'Log-Distance',
    'MAE': round(mean_absolute_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['Log-Distance']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['Log-Distance']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['Log-Distance']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['Log-Distance'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos_CM = pd.concat([desempenho_modelos_CM, pd.DataFrame({
    'modelo': 'Log-Distance Multi-Exponent',
    'MAE': round(mean_absolute_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['log_distance_ME']), 2),
    'MAPE': round(mean_absolute_percentage_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['log_distance_ME']), 2),
    'MSE': round(mean_squared_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['log_distance_ME']), 2),
    'RMSE': round(math.sqrt(mean_squared_error(enlaces_cobertura_mista['PL-FC'], enlaces_cobertura_mista['log_distance_ME'])), 2)
}, index=[0])], ignore_index=True)

desempenho_modelos_CM

,modelo,MAE,MAPE,MSE,RMSE
0,FSPL,5.20,0.07,39.40,6.28
1,Log-Distance,3.91,0.05,15.66,3.96
2,Log-Distance Multi-Exponent,2.99,0.04,12.19,3.49
